In [1]:
from sklearn.metrics import accuracy_score, mean_squared_error, roc_curve, auc

In [2]:
%matplotlib inline
import pandas as pd

df = pd.read_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/train.csv')
print(df.head(5))

   Id  Popularity                                       Page content
0   0          -1  <html><head><div class="article-info"> <span c...
1   1           1  <html><head><div class="article-info"><span cl...
2   2           1  <html><head><div class="article-info"><span cl...
3   3          -1  <html><head><div class="article-info"><span cl...
4   4          -1  <html><head><div class="article-info"><span cl...


In [3]:
testdata = pd.read_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/test.csv')
print(df.head(5))

   Id  Popularity                                       Page content
0   0          -1  <html><head><div class="article-info"> <span c...
1   1           1  <html><head><div class="article-info"><span cl...
2   2           1  <html><head><div class="article-info"><span cl...
3   3          -1  <html><head><div class="article-info"><span cl...
4   4          -1  <html><head><div class="article-info"><span cl...


In [4]:
print(df.loc[0,'Page content'])

<html><head><div class="article-info"> <span class="byline basic">Clara Moskowitz</span> for <a href="/publishers/space-com/">Space.com</a> <time datetime="Wed, 19 Jun 2013 15:04:30 +0000">2013-06-19 15:04:30 UTC</time> </div></head><body><h1 class="title">NASA's Grand Challenge: Stop Asteroids From Destroying Earth</h1><figure class="article-image"><img class="microcontent" data-fragment="lead-image" data-image="http://i.amz.mshcdn.com/I7b9cUsPSztew7r1WT6_iBLjflo=/950x534/2013%2F06%2F19%2Ffe%2FDactyl.44419.jpg" data-micro="1" data-url="http://mashable.com/2013/06/19/nasa-grand-challenge-asteroid/" src="http://i.amz.mshcdn.com/I7b9cUsPSztew7r1WT6_iBLjflo=/950x534/2013%2F06%2F19%2Ffe%2FDactyl.44419.jpg"/></figure><article data-channel="world"><section class="article-content"> <p>There may be killer asteroids headed for Earth, and NASA has decided to do something about it. The space agency announced a new "Grand Challenge" on June 18 to find all dangerous space rocks and figure out how t

In [5]:
import re
from bs4 import BeautifulSoup

def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    #r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    #emoticons = re.findall(r, text)
    #text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    #text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    text = re.sub('[\W]+', ' ', text.lower()) + ' '
    return text

In [6]:
preprocessor(df.loc[0,'Page content'])

' clara moskowitz for space com 2013 06 19 15 04 30 utc nasa s grand challenge stop asteroids from destroying earth there may be killer asteroids headed for earth and nasa has decided to do something about it the space agency announced a new grand challenge on june 18 to find all dangerous space rocks and figure out how to stop them from destroying our planet the new mission builds on projects already underway at nasa including a plan to capture an asteroid pull it in toward the moon and send astronauts to visit it as part of the grand challenge the agency issued a request for information today aiming to solicit ideas from industry academia and the public on how to improve the asteroid mission plan we re asking for you to think about concepts and different approaches for what we ve described here william gerstenmaier nasa s associate administrator for human explorations and operations said yesterday during a nasa event announcing the initiative we want you to think about other ways of 

## Preprocessing: Stop-Word Removal

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

print(tokenizer_stem_nostop('runners like running and thus they run'))

['runner', 'like', 'run', 'thu', 'run']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stat_pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## BoW (Bag-Of-Words)

In [8]:
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer

doc_dummy = ["Study hard, then you will be happy and I will be happy", 
           "\"I'm not happy :(\" \", because you don't study hard"]
print('[example documents]\n{}\n'.format('\n'.join(doc_dummy)))

# ngram_range=(min,max), default: 1-gram => (1,1)
count = CountVectorizer(ngram_range=(1, 1),
                        preprocessor=preprocessor,
                        tokenizer=tokenizer_stem_nostop)

count.fit([df.loc[0,'Page content']])
# dictionary is stored in vocabulary_
BoW = count.vocabulary_
print('[vocabulary]\n{}'.format(BoW))


[example documents]
Study hard, then you will be happy and I will be happy
"I'm not happy :(" ", because you don't study hard

[vocabulary]
{'clara': 33, 'moskowitz': 129, 'space': 181, 'com': 35, 'utc': 209, 'nasa': 131, 'grand': 88, 'challeng': 29, 'stop': 188, 'asteroid': 16, 'destroy': 55, 'earth': 65, 'may': 122, 'killer': 110, 'head': 90, 'decid': 49, 'someth': 180, 'agenc': 4, 'announc': 10, 'new': 132, 'june': 108, 'find': 77, 'danger': 46, 'rock': 163, 'figur': 76, 'planet': 147, 'mission': 124, 'build': 22, 'project': 151, 'alreadi': 6, 'underway': 207, 'includ': 100, 'plan': 146, 'captur': 25, 'pull': 158, 'toward': 204, 'moon': 128, 'send': 172, 'astronaut': 17, 'visit': 213, 'part': 142, 'issu': 106, 'request': 160, 'inform': 102, 'today': 200, 'aim': 5, 'solicit': 178, 'idea': 96, 'industri': 101, 'academia': 1, 'public': 156, 'improv': 99, 'ask': 14, 'think': 196, 'concept': 37, 'differ': 58, 'approach': 12, 'describ': 53, 'william': 219, 'gerstenmai': 84, 'associ': 15, 

C:\Users\stat_pc\anaconda3\envs\python3.7.9v1\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


## TF-IDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,1),
                        preprocessor=preprocessor,
                        tokenizer=tokenizer_stem_nostop)


## Feature Hashing

In [10]:
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import HashingVectorizer

print('[example documents]\n{}\n'.format('\n'.join(doc_dummy)))

# hash words to 1024 buckets
hashvec = HashingVectorizer(n_features=2**10,
                            preprocessor=preprocessor,
                            tokenizer=tokenizer_stem_nostop)


[example documents]
Study hard, then you will be happy and I will be happy
"I'm not happy :(" ", because you don't study hard



# Logistic Regression pipe3 Kaggle 0.53452

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

In [9]:
# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['LogisticRegression', 
         'LogisticRegression+(1,2)gram',
         'LogisticRegression+preprocess',
         'LogisticRegression+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', LogisticRegression(solver = "liblinear",max_iter=10000))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', LogisticRegression(solver = "liblinear",max_iter=10000))])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', LogisticRegression(solver = "liblinear",max_iter=10000))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', LogisticRegression(solver = "liblinear",max_iter=10000))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2, pipe3, pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    6.8s remaining:    6.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    7.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


LogisticRegression: 0.487 (+/-0.056)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.2min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.3min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


LogisticRegression+(1,2)gram: 0.487 (+/-0.047)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   17.5s remaining:   17.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   17.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


LogisticRegression+preprocess: 0.515 (+/-0.048)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   18.1s remaining:   18.1s


LogisticRegression+preprocess+hash: 0.509 (+/-0.067)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   18.4s finished


In [10]:
pipe3.fit(df['Page content'],df['Popularity'])

Pipeline(steps=[('vect',
                 TfidfVectorizer(preprocessor=<function preprocessor at 0x000001A8F7E1F8B8>,
                                 tokenizer=<function tokenizer_stem_nostop at 0x000001A89CB631F8>)),
                ('clf',
                 LogisticRegression(max_iter=10000, solver='liblinear'))])

In [12]:
pred = pipe3.predict(testdata['Page content'])
pred[pred==-1] = 0

In [27]:
pred_csv= np.zeros((testdata.shape[0],2))
pred_csv = pd.DataFrame(pred_csv)
pred_csv.columns = ['Id','Popularity']
pred_csv['Id'] = testdata['Id']
pred_csv['Popularity'] = pred
pd.DataFrame(pred_csv).to_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/y_pred.csv',index=False,header=True)


## Xgboost pipe1 Kaggle 0.54126

In [12]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

In [30]:
# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram',
         'XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier())])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', XGBClassifier())])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier())])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier())])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2, pipe3, pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   18.4s remaining:   18.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   18.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.537 (+/-0.044)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  4.1min remaining:  4.1min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+(1,2)gram: 0.532 (+/-0.049)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   23.5s remaining:   23.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   24.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+preprocess: 0.495 (+/-0.059)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   17.9s remaining:   17.9s


XGBClassifier+preprocess+hash: 0.502 (+/-0.038)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   18.2s finished


In [31]:
pipe1.fit(df['Page content'],df['Popularity'])

Pipeline(steps=[('vect', CountVectorizer()), ('clf', XGBClassifier())])

In [33]:
pred = pipe1.predict(testdata['Page content'])
pred[pred==-1] = 0

In [34]:
pred_csv= np.zeros((testdata.shape[0],2))
pred_csv = pd.DataFrame(pred_csv)
pred_csv.columns = ['Id','Popularity']
pred_csv['Id'] = testdata['Id']
pred_csv['Popularity'] = pred
pd.DataFrame(pred_csv).to_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/y_pred_v2.csv',index=False,header=True)


# SVM

In [36]:
from sklearn import svm
# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram',
         'XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', svm.SVC())])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', svm.SVC())])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', svm.SVC())])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', svm.SVC())])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2, pipe3, pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    6.3s remaining:    6.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.506 (+/-0.066)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   25.2s remaining:   25.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   26.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+(1,2)gram: 0.504 (+/-0.065)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   18.1s remaining:   18.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   18.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+preprocess: 0.506 (+/-0.051)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   17.8s remaining:   17.8s


XGBClassifier+preprocess+hash: 0.510 (+/-0.065)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   18.2s finished


# Naive Bayes

In [37]:
from sklearn.naive_bayes import GaussianNB
# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['GaussianNB', 
         'GaussianNB+(1,2)gram',
         'GaussianNB+preprocess',
         'GaussianNB+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', GaussianNB())])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', GaussianNB())])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', GaussianNB())])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', GaussianNB())])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2, pipe3, pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    2.9s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    3.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


GaussianNB: nan (+/-nan)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   12.7s remaining:   12.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   13.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


GaussianNB+(1,2)gram: nan (+/-nan)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   14.3s remaining:   14.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   14.5s finished


GaussianNB+preprocess: nan (+/-nan)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


GaussianNB+preprocess+hash: nan (+/-nan)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   14.3s remaining:   14.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   14.5s finished


In [39]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram']

pipe1 = Pipeline([('vect', CountVectorizer(ngram_range=(1,5),preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier())])

pipe2 = Pipeline([('vect', HashingVectorizer(n_features=2**20,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier())])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  4.5min remaining:  4.5min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.496 (+/-0.067)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  4.3min remaining:  4.3min


XGBClassifier+(1,2)gram: 0.467 (+/-0.063)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.4min finished


# Xgboost pipe2 Kaggle 0.54907

In [40]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram',
         'XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =50, random_state= 0,subsample=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =50, random_state= 0,subsample=0.5))])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =50, random_state= 0,subsample=0.5))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =50, random_state= 0,subsample=0.5))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2, pipe3, pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   16.8s remaining:   16.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   17.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.544 (+/-0.050)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  3.6min remaining:  3.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+(1,2)gram: 0.549 (+/-0.052)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   22.3s remaining:   22.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   22.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+preprocess: 0.510 (+/-0.047)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   18.3s remaining:   18.3s


XGBClassifier+preprocess+hash: 0.516 (+/-0.040)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   18.5s finished


In [22]:
# fit model花了40分鐘左右...
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =50, random_state= 0,subsample=0.5))])

pipe2.fit(df['Page content'],df['Popularity'])

pred = pipe2.predict(testdata['Page content'])
pred[pred==-1] = 0

pred_csv= np.zeros((testdata.shape[0],2))
pred_csv = pd.DataFrame(pred_csv)
pred_csv.columns = ['Id','Popularity']
pred_csv['Id'] = testdata['Id']
pred_csv['Popularity'] = pred
pd.DataFrame(pred_csv).to_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/y_pred_v3.csv',index=False,header=True)

# Xgboost pipe2 Kaggle 0.57054

In [25]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,5)gram']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,5))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   30.3s remaining:   30.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   30.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.560 (+/-0.046)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 16.7min remaining: 16.7min


XGBClassifier+(1,5)gram: 0.564 (+/-0.041)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 16.8min finished


In [17]:
#14:30~17:54
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,5))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,n_jobs=-1,verbose=3))])
pipe2.fit(df['Page content'],df['Popularity'])

pred = pipe2.predict_proba(testdata['Page content'])[:,1]

In [19]:
pred_csv= np.zeros((testdata.shape[0],2))
pred_csv = pd.DataFrame(pred_csv)
pred_csv.columns = ['Id','Popularity']
pred_csv['Id'] = testdata['Id']
pred_csv['Popularity'] = pred
pd.DataFrame(pred_csv).to_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/y_pred_v5.csv',index=False,header=True)

# Xgboost 

In [15]:
# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe3, pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))


[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   33.0s remaining:   33.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   33.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+preprocess: 0.509 (+/-0.048)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   20.8s remaining:   20.8s


XGBClassifier+preprocess+hash: 0.504 (+/-0.038)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   21.6s finished


In [16]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,5)gram']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =150, random_state= 0,subsample=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,5))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =150, random_state= 0,subsample=0.5))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   46.5s remaining:   46.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   46.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.559 (+/-0.040)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 25.1min remaining: 25.1min


XGBClassifier+(1,5)gram: 0.565 (+/-0.044)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 25.3min finished


In [17]:
## depth

from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,5)gram']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 10, min_child_weight = 1e-05, n_estimators =150, random_state= 0,subsample=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,5))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 10, min_child_weight = 1e-05, n_estimators =150, random_state= 0,subsample=0.5))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.5min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.549 (+/-0.043)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 50.2min remaining: 50.2min


XGBClassifier+(1,5)gram: 0.556 (+/-0.038)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 50.5min finished


In [18]:
## depth

from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,5)gram']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer(preprocessor=preprocessor)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(preprocessor=preprocessor,ngram_range=(1,5))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   24.6s remaining:   24.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   24.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.560 (+/-0.028)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 11.7min remaining: 11.7min


XGBClassifier+(1,5)gram: 0.548 (+/-0.033)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 11.8min finished


In [22]:
## depth

from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,5)gram']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 0.5, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,10))), 
                  ('clf', XGBClassifier(colsample_bytree = 0.5, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   17.4s remaining:   17.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   17.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.558 (+/-0.034)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 91.0min remaining: 91.0min


XGBClassifier+(1,5)gram: 0.550 (+/-0.047)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 92.1min finished


In [11]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,1))), 
                  ('clf', XGBClassifier())])
df_small = df.sample(n=1000, random_state=0)
pipe2.fit(df_small['Page content'],df_small['Popularity'])

#pred = pipe2.predict(testdata['Page content'])


Pipeline(steps=[('vect', CountVectorizer()), ('clf', XGBClassifier())])

In [16]:
pipe2.predict_proba(df_small['Page content'])[:,1]

array([0.23694882, 0.43114597, 0.6280522 , 0.29912624, 0.43623105,
       0.38536134, 0.7120188 , 0.76651555, 0.7586255 , 0.5573365 ,
       0.51771253, 0.37978962, 0.6109432 , 0.7578659 , 0.7346292 ,
       0.48538724, 0.38706902, 0.72815496, 0.30742812, 0.4450541 ,
       0.72462904, 0.36421612, 0.6487742 , 0.702284  , 0.30932677,
       0.6284739 , 0.46544173, 0.23199305, 0.2529717 , 0.2797301 ,
       0.6262077 , 0.7459523 , 0.3049826 , 0.67328054, 0.7114652 ,
       0.21217795, 0.3653934 , 0.33806783, 0.43076172, 0.39572752,
       0.39802375, 0.67044246, 0.8128762 , 0.29590946, 0.34572393,
       0.47156206, 0.51480734, 0.34743178, 0.6768711 , 0.30221355,
       0.23471804, 0.68349653, 0.5715238 , 0.6891289 , 0.6350345 ,
       0.7238575 , 0.31274128, 0.6308068 , 0.42847425, 0.74643314,
       0.7475513 , 0.36119676, 0.1928986 , 0.81937766, 0.26104608,
       0.69364357, 0.42908227, 0.18135762, 0.46714747, 0.3988037 ,
       0.51061916, 0.39538565, 0.41403446, 0.31881848, 0.34373

## Randomforest

In [21]:
from sklearn.ensemble import RandomForestClassifier

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram',
         'XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', RandomForestClassifier())])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', RandomForestClassifier(n_estimators=150,max_dept=5,min_samples_split=1,min_samples_leaf=0.5,n_jobs=-1,verbose=3))])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', RandomForestClassifier(n_estimators=150,max_dept=5,min_samples_split=1,min_samples_leaf=0.5,n_jobs=-1,verbose=3))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', RandomForestClassifier(n_estimators=150,max_dept=5,min_samples_split=1,min_samples_leaf=0.5,n_jobs=-1,verbose=3))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2, pipe3, pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    6.8s remaining:    6.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.542 (+/-0.039)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   22.6s remaining:   22.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   23.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+(1,2)gram: 0.530 (+/-0.037)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   17.5s remaining:   17.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   17.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+preprocess: 0.503 (+/-0.041)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   17.1s remaining:   17.1s


XGBClassifier+preprocess+hash: 0.498 (+/-0.041)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   17.3s finished


In [23]:
from sklearn.ensemble import RandomForestClassifier

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram',
         'XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', RandomForestClassifier())])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', RandomForestClassifier(n_estimators=150,max_depth=5,min_samples_split=1,min_samples_leaf=0.5,n_jobs=-1,verbose=3))])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', RandomForestClassifier(n_estimators=150,max_depth=5,min_samples_split=1,min_samples_leaf=0.5,n_jobs=-1,verbose=3))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', RandomForestClassifier(n_estimators=150,max_depth=5,min_samples_split=1,min_samples_leaf=0.5,n_jobs=-1,verbose=3))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2, pipe3, pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    5.9s remaining:    5.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.539 (+/-0.027)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   13.8s remaining:   13.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   14.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+(1,2)gram: nan (+/-nan)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   15.3s remaining:   15.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   15.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+preprocess: nan (+/-nan)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   14.9s remaining:   14.9s


XGBClassifier+preprocess+hash: nan (+/-nan)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   15.3s finished


In [28]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram',
         'XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 0.5, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.7))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,5))), 
                  ('clf', from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,5)gram']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,5))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))(colsample_bytree = 0.5, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.7))])
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

SyntaxError: invalid syntax (<ipython-input-28-f925e31ae2e4>, line 17)

# Xgboost pipe2 Kaggle 0.56985

In [12]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,5)gram']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=1))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,5))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=1))])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=1))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=1))])
# CV

print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2,pipe3,pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

NameError: name 'Pipeline' is not defined

In [32]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,5)gram']
# without preprocessing
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=1))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=1))])
# CV

print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe3,pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   42.6s remaining:   42.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   43.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.519 (+/-0.029)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   21.2s remaining:   21.2s


XGBClassifier+(1,5)gram: 0.540 (+/-0.062)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   21.9s finished


In [ ]:
#19:46~23:02
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,5))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=1))])
pipe2.fit(df['Page content'],df['Popularity'])

pred = pipe2.predict_proba(testdata['Page content'])[:,1]

pred_csv= np.zeros((testdata.shape[0],2))
pred_csv = pd.DataFrame(pred_csv)
pred_csv.columns = ['Id','Popularity']
pred_csv['Id'] = testdata['Id']
pred_csv['Popularity'] = pred
pd.DataFrame(pred_csv).to_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/y_pred_v6.csv',index=False,header=True)

# Xgboost

In [33]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,5)gram']

pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# CV
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe3, pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   31.4s remaining:   31.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   31.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.562 (+/-0.058)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 17.3min remaining: 17.3min


XGBClassifier+(1,5)gram: 0.570 (+/-0.031)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 17.4min finished


In [36]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,5)gram']

pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 10, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 10, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5))])
# CV
# CV
print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe3, pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.0min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.0min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.510 (+/-0.048)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   22.7s remaining:   22.7s


XGBClassifier+(1,5)gram: 0.538 (+/-0.053)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   22.9s finished


# Xgboost pipe1 0.56441

In [37]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram',
         'XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,5))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# CV

print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2,pipe3,pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   33.2s remaining:   33.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   33.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.564 (+/-0.060)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 17.3min remaining: 17.3min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 17.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+(1,2)gram: 0.570 (+/-0.030)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   41.0s remaining:   41.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   41.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+preprocess: 0.517 (+/-0.030)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   22.2s remaining:   22.2s


XGBClassifier+preprocess+hash: 0.540 (+/-0.062)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   22.9s finished


In [38]:
import time
t1 = time.time()

pipe1.fit(df['Page content'],df['Popularity'])

t2 = time.time()
print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')


t1 = time.time()

pred = pipe1.predict_proba(testdata['Page content'])[:,1]

t2 = time.time()
print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')


pred_csv= np.zeros((testdata.shape[0],2))
pred_csv = pd.DataFrame(pred_csv)
pred_csv.columns = ['Id','Popularity']
pred_csv['Id'] = testdata['Id']
pred_csv['Popularity'] = pred
pd.DataFrame(pred_csv).to_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/y_pred_v7.csv',index=False,header=True)


time elapsed: 247.28 seconds
time elapsed: 13.64 seconds


# Xgboost pip2 0.57007

In [11]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)

names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram',
         'XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# CV

print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2,pipe3,pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   33.1s remaining:   33.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   33.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.564 (+/-0.060)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  7.1min remaining:  7.1min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+(1,2)gram: 0.569 (+/-0.033)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   30.8s remaining:   30.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   31.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+preprocess: 0.516 (+/-0.040)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   21.4s remaining:   21.4s


XGBClassifier+preprocess+hash: 0.511 (+/-0.040)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   21.9s finished


In [12]:
import time
t1 = time.time()

pipe2.fit(df['Page content'],df['Popularity'])

t2 = time.time()
print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')


t1 = time.time()

pred = pipe2.predict_proba(testdata['Page content'])[:,1]

t2 = time.time()
print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')


pred_csv= np.zeros((testdata.shape[0],2))
pred_csv = pd.DataFrame(pred_csv)
pred_csv.columns = ['Id','Popularity']
pred_csv['Id'] = testdata['Id']
pred_csv['Popularity'] = pred
pd.DataFrame(pred_csv).to_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/y_pred_v8.csv',index=False,header=True)

time elapsed: 4088.09 seconds
time elapsed: 48.61 seconds


# Xgboost

In [16]:
y = np.array(df['Popularity']) 
y[y==-1]=0
df['Popularity'] = y

In [17]:
df['Popularity']

0        0
1        1
2        1
3        0
4        0
        ..
27638    0
27639    0
27640    0
27641    0
27642    1
Name: Popularity, Length: 27643, dtype: int64

In [18]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)
df_small['Popularity']
names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram',
         'XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 5, min_child_weight = 1e-05, n_estimators =100, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# CV

print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2,pipe3,pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   35.8s remaining:   35.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   36.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.564 (+/-0.060)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  7.2min remaining:  7.2min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.3min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+(1,2)gram: 0.569 (+/-0.033)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   30.8s remaining:   30.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   31.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+preprocess: 0.516 (+/-0.040)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   21.4s remaining:   21.4s


XGBClassifier+preprocess+hash: 0.511 (+/-0.040)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   22.4s finished


In [19]:
df_small['Popularity']

13516    0
11607    0
11866    1
25540    0
26611    0
        ..
12673    0
14569    0
22266    1
19962    0
17686    1
Name: Popularity, Length: 1000, dtype: int64

In [32]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# randomly sample 1000 examples
df_small = df.sample(n=1000, random_state=0)
df_small['Popularity']
names = ['XGBClassifier', 
         'XGBClassifier+(1,2)gram',
         'XGBClassifier+preprocess',
         'XGBClassifier+preprocess+hash']
# without preprocessing
pipe1 = Pipeline([('vect', CountVectorizer()), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 15, min_child_weight = 1e-05, n_estimators =150, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# without preprocessing, use larger ngram range
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 15, min_child_weight = 1e-05, n_estimators =150, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# with preprocessing
pipe3 = Pipeline([('vect', TfidfVectorizer(preprocessor=preprocessor, 
                                           tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 15, min_child_weight = 1e-05, n_estimators =150, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# with preprocessing and hasing
pipe4 = Pipeline([('vect', HashingVectorizer(n_features=2**10,
                                             preprocessor=preprocessor, 
                                             tokenizer=tokenizer_stem_nostop)), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 15, min_child_weight = 1e-05, n_estimators =150, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])
# CV

print('[auc (10-fold cv)]')
for name, clf in zip(names, [pipe1, pipe2,pipe3,pipe4]):
    scores = cross_val_score(estimator=clf, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
    print('%s: %.3f (+/-%.3f)' % (name, scores.mean(), scores.std()))

[auc (10-fold cv)]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  2.1min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier: 0.551 (+/-0.046)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 31.6min remaining: 31.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 31.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+(1,2)gram: 0.567 (+/-0.042)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.2min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier+preprocess: 0.514 (+/-0.047)


[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   31.0s remaining:   31.0s


XGBClassifier+preprocess+hash: 0.506 (+/-0.035)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   31.6s finished


In [ ]:
df_small = df.sample(n=3000, random_state=0)
scores = cross_val_score(estimator=pipe2, X=df_small['Page content'], y=df_small['Popularity'], \
                         cv=10, scoring='roc_auc',n_jobs=-1,verbose=2)
print('%s: %.3f (+/-%.3f)' % ('XGBClassifier+(1,3)gram', scores.mean(), scores.std()))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


# Xgboost

In [14]:
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 15, min_child_weight = 1e-05, n_estimators =150, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])

import time
t1 = time.time()

pipe2.fit(df['Page content'],df['Popularity'])

t2 = time.time()
print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')


t1 = time.time()

pred = pipe2.predict_proba(testdata['Page content'])[:,1]

t2 = time.time()
print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')


pred_csv= np.zeros((testdata.shape[0],2))
pred_csv = pd.DataFrame(pred_csv)
pred_csv.columns = ['Id','Popularity']
pred_csv['Id'] = testdata['Id']
pred_csv['Popularity'] = pred
pd.DataFrame(pred_csv).to_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/y_pred_v9.csv',index=False,header=True)

time elapsed: 21354.36 seconds
time elapsed: 45.39 seconds


# Xgboost

In [16]:
pipe2 = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), 
                  ('clf', XGBClassifier(colsample_bytree = 1, learning_rate = 0.001, max_depth = 20, min_child_weight = 1e-05, n_estimators =150, random_state= 0,subsample=0.5,max_delta_step=1,gamma=0.5))])

import time
t1 = time.time()

pipe2.fit(df['Page content'],df['Popularity'])

t2 = time.time()
print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')


t1 = time.time()

pred = pipe2.predict_proba(testdata['Page content'])[:,1]

t2 = time.time()
print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')


pred_csv= np.zeros((testdata.shape[0],2))
pred_csv = pd.DataFrame(pred_csv)
pred_csv.columns = ['Id','Popularity']
pred_csv['Id'] = testdata['Id']
pred_csv['Popularity'] = pred
pd.DataFrame(pred_csv).to_csv('C:/Users/stat_pc/Desktop/深度學習/Competition 01/y_pred_v10.csv',index=False,header=True)

time elapsed: 29292.52 seconds
time elapsed: 49.86 seconds
